<a href="https://colab.research.google.com/github/wenchun0731/github-colab/blob/try/yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import os
import numpy

os.mkdir("/content/plate_train_data")
os.mkdir("/content/plate_train_data/images")
os.mkdir("/content/plate_train_data/labels")
os.mkdir("/content/plate_train_data/images/train")
os.mkdir("/content/plate_train_data/images/val")
os.mkdir("/content/plate_train_data/labels/train")
os.mkdir("/content/plate_train_data/labels/val")

In [4]:
import xml.etree.ElementTree as ET
from xml.dom.minidom import parse

path = "/content/gdrive/MyDrive/AI/dataset/CarLicensePlateDetection/annotations"
classes = {"licence":0}

for annotations in os.listdir(path):
    dom = parse(os.path.join(path,annotations))
    root = dom.documentElement
    filename = ".txt".join(root.getElementsByTagName("filename")[0].childNodes[0].data.split(".png"))
    image_width = root.getElementsByTagName("width")[0].childNodes[0].data
    image_height = root.getElementsByTagName("height")[0].childNodes[0].data
    with open("/content/plate_train_data/labels/train/"+filename,"w") as r:
        for items in root.getElementsByTagName("object") :
            name = items.getElementsByTagName("name")[0].childNodes[0].data
            xmin = items.getElementsByTagName("xmin")[0].childNodes[0].data
            ymin = items.getElementsByTagName("ymin")[0].childNodes[0].data
            xmax = items.getElementsByTagName("xmax")[0].childNodes[0].data
            ymax = items.getElementsByTagName("ymax")[0].childNodes[0].data
            x_center_norm = ((int(xmin)+int(xmax)) / 2 ) / int(image_width)
            y_center_norm = ((int(ymin)+int(ymax))/2) / int(image_height)
            width_norm = ((int(xmax)-int(xmin))/int(image_width))
            height_norm = ((int(ymax)-int(ymin))/int(image_height))
            r.write(str(classes[name])+" ")
            r.write(str(x_center_norm)+" ")
            r.write(str(y_center_norm)+" ")
            r.write(str(width_norm)+" ")
            r.write(str(height_norm)+"\n")

In [5]:
import shutil
path = "/content/gdrive/MyDrive/AI/dataset/CarLicensePlateDetection/images"

for images in os.listdir(path):
  image_path = os.path.join(path,images)
  shutil.copy(image_path,"/content/plate_train_data/images/train/"+images)

In [6]:
from random import shuffle
from glob import glob
import random
import shutil

random.seed(42)
files = glob("/content/plate_train_data/images/train/*.png")
shuffle(files)
txt_train_path = "/content/plate_train_data/labels/train/"
img_train_path = "/content/plate_train_data/images/train/"
txt_val_path = "/content/plate_train_data/labels/val/"
img_val_path = "/content/plate_train_data/images/val/"
for i in files[:120]:
    filename = i.split("/")[-1]
    shutil.move(txt_train_path+filename.replace(".png",".txt"),\
                txt_val_path+filename.replace(".png",".txt"))
    shutil.move(img_train_path+filename,img_val_path+filename)

In [7]:
%cd /content
!echo "train: /content/plate_train_data/images/train" > licence.yaml
!echo "val:   /content/plate_train_data/images/val"  >> licence.yaml
!echo "nc : 1" >> licence.yaml
!echo "names: ['licence']" >> licence.yaml

/content


In [8]:
%cd /content/
%rm -rf yolov5
!git clone https://github.com/ultralytics/yolov5


/content
Cloning into 'yolov5'...
remote: Enumerating objects: 16517, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 16517 (delta 47), reused 55 (delta 17), pack-reused 16402
Receiving objects: 100% (16517/16517), 15.12 MiB | 20.93 MiB/s, done.
Resolving deltas: 100% (11309/11309), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00


In [ ]:
%cd yolov5
!pip install -r requirements.txt

In [9]:
!python train.py --weights yolov5s.pt --cfg models/yolov5s.yaml --img 416 --batch 48 --epochs 40 --data /content/licence.yaml  --cache

2024-03-12 03:06:25.544647: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 03:06:25.544712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 03:06:25.546062: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=/content/licence.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=48, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=

In [10]:
!python detect.py --weights runs/train/exp/weights/best.pt --conf 0.25 --img-size 640 --source /content/car56.jpg

detect: weights=['runs/train/exp/weights/best.pt'], source=/content/car56.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1 /content/car56.jpg: 448x640 1 licence, 48.9ms
Speed: 0.5ms pre-process, 48.9ms inference, 635.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp
